<a href="https://colab.research.google.com/github/sre-glitch/Bird_species_analysis/blob/main/Bird_species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROJECT-**
## **Bird Species Observation Analysis**
***By Sreya Tulasi Kolasani***

# **GitHub Link:**

# **Problem Statement:**
The project aims to analyze the distribution and diversity of bird species in two distinct ecosystems: forests and grasslands. By examining bird species observations across these habitats, the goal is to understand how environmental factors, such as vegetation type, climate, and terrain, influence bird populations and their behavior. The study will involve working on the provided observational data of bird species present in both ecosystems, identifying patterns of habitat preference, and assessing the impact of these habitats on bird diversity. The findings can provide valuable insights into habitat conservation, biodiversity management, and the effects of environmental changes on avian communities.


# **Import the files**

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Bird_Monitoring_Data_GRASSLAND.XLSX to Bird_Monitoring_Data_GRASSLAND (1).XLSX
Saving Bird_Monitoring_Data_FOREST.XLSX to Bird_Monitoring_Data_FOREST (1).XLSX


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

In [ ]:
!pip install plotly openpyxl

In [ ]:
# Load all sheets from the Forest dataset
forest_data = pd.read_excel("Bird_Monitoring_Data_FOREST.XLSX", sheet_name=None)
grassland_data = pd.read_excel("Bird_Monitoring_Data_GRASSLAND.XLSX", sheet_name=None)

# Combine all sheets into one DataFrame
df_forest = pd.concat(forest_data.values(), ignore_index=True)
df_grassland = pd.concat(grassland_data.values(), ignore_index=True)

/tmp/ipython-input-4004321372.py:7: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_grassland = pd.concat(grassland_data.values(), ignore_index=True)


In [ ]:
# Add a Habitat column
df_forest['Habitat'] = 'Forest'
df_grassland['Habitat'] = 'Grassland'

# Combine both
df = pd.concat([df_forest, df_grassland], ignore_index=True)

In [ ]:
# Check missing data summary
missing_summary = df.isnull().sum()
missing_percent = (missing_summary / len(df)) * 100
missing_df = pd.DataFrame({'Missing Values': missing_summary, 'Percent': missing_percent})
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values(by='Percent', ascending=False)
missing_df

,Missing Values,Percent
Sub_Unit_Code,16355,95.772091
TaxonCode,8548,50.055630
Previously_Obs,8546,50.043919
Site_Name,8531,49.956081
NPSTaxonCode,8531,49.956081
Sex,5183,30.350764
Distance,1486,8.701763
AcceptedTSN,33,0.193242
ID_Method,2,0.011712


In [ ]:
# Drop critical missing data
df.dropna(subset=['Scientific_Name', 'Date', 'Plot_Name'], inplace=True)

# Fill environmental columns with median
for col in ['Temperature', 'Humidity']:
    df[col] = df[col].fillna(df[col].median())

# Optional: Fill categorical with mode
categorical_cols = ['Sky', 'Wind', 'Disturbance', 'Sex', 'ID_Method']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [ ]:
# Ensure datetime is in correct format
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Add temporal features
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Season'] = df['Month'].apply(lambda x: 'Winter' if x in [12,1,2] else
                                             'Spring' if x in [3,4,5] else
                                             'Summer' if x in [6,7,8] else 'Fall')

# Clean 'Sex' column to use consistent values
df['Sex'] = df['Sex'].str.title().replace({'Undet': 'Undetermined', 'U': 'Undetermined'})

# Clean boolean-like columns
df['Flyover_Observed'] = df['Flyover_Observed'].astype(str).str.upper().replace({'TRUE': True, 'FALSE': False})
df['PIF_Watchlist_Status'] = df['PIF_Watchlist_Status'].astype(str).str.upper().replace({'TRUE': True, 'FALSE': False})
df['Regional_Stewardship_Status'] = df['Regional_Stewardship_Status'].astype(str).str.upper().replace({'TRUE': True, 'FALSE': False})

/tmp/ipython-input-416697430.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Flyover_Observed'] = df['Flyover_Observed'].astype(str).str.upper().replace({'TRUE': True, 'FALSE': False})
/tmp/ipython-input-416697430.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PIF_Watchlist_Status'] = df['PIF_Watchlist_Status'].astype(str).str.upper().replace({'TRUE': True, 'FALSE': False})
/tmp/ipython-input-416697430.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. 

In [ ]:
# Keep only relevant columns
columns_to_keep = [
    'Admin_Unit_Code', 'Site_Name', 'Plot_Name', 'Location_Type', 'Habitat',
    'Date', 'Year', 'Month', 'Season', 'Start_Time', 'End_Time', 'Observer', 'Visit',
    'Interval_Length', 'ID_Method', 'Distance', 'Flyover_Observed', 'Sex',
    'Common_Name', 'Scientific_Name', 'AcceptedTSN', 'NPSTaxonCode', 'AOU_Code',
    'PIF_Watchlist_Status', 'Regional_Stewardship_Status',
    'Temperature', 'Humidity', 'Sky', 'Wind', 'Disturbance',
    'Initial_Three_Min_Cnt'
]

df = df[columns_to_keep]

In [ ]:
# Preview cleaned dataset
df.info()

# Save cleaned data (optional)
df.to_csv("cleaned_bird_data.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17077 entries, 0 to 17076
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Admin_Unit_Code              17077 non-null  object        
 1   Site_Name                    8546 non-null   object        
 2   Plot_Name                    17077 non-null  object        
 3   Location_Type                17077 non-null  object        
 4   Habitat                      17077 non-null  object        
 5   Date                         17077 non-null  datetime64[ns]
 6   Year                         17077 non-null  int32         
 7   Month                        17077 non-null  int32         
 8   Season                       17077 non-null  object        
 9   Start_Time                   17077 non-null  object        
 10  End_Time                     17077 non-null  object        
 11  Observer                     17077 non-nu

***Distribution of Species Across Administrative Units and Habitat Types:***

In [ ]:
# Species count per Admin Unit
admin_species = df.groupby('Admin_Unit_Code')['Scientific_Name'].nunique().sort_values(ascending=False)
print(admin_species)

# Species count per Habitat
habitat_species = df.groupby('Habitat')['Scientific_Name'].nunique()
print(habitat_species)

Admin_Unit_Code
MONO    100
MANA     81
ANTI     81
CHOH     80
NACE     66
HAFE     55
PRWI     54
GWMP     49
CATO     46
ROCR     45
WOTR     27
Name: Scientific_Name, dtype: int64
Habitat
Forest       108
Grassland    107
Name: Scientific_Name, dtype: int64


# **Temporal Analysis**
***SEASONAL TRENDS: Observation Frequency by Year, Month, and Season***

In [ ]:
# Yearly observations
yearly_obs = df['Year'].value_counts().sort_index()
print(yearly_obs)

# Monthly observations
monthly_obs = df['Month'].value_counts().sort_index()
print(monthly_obs)

# Seasonal observations
seasonal_obs = df['Season'].value_counts()
print(seasonal_obs)

Year
2018    17077
Name: count, dtype: int64
Month
5    5596
6    6596
7    4885
Name: count, dtype: int64
Season
Summer    11481
Spring     5596
Name: count, dtype: int64


***OBSERVATION TIME ANALYSIS: to determine if specific time windows correlate with higher bird activity***

In [ ]:
# Convert Start_Time and End_Time to datetime if needed (example)
df['Start_Time'] = pd.to_datetime(df['Start_Time'], errors='coerce').dt.time
df['End_Time'] = pd.to_datetime(df['End_Time'], errors='coerce').dt.time

# Explore time ranges with most bird activity
activity_by_time = df.groupby('Start_Time')['Scientific_Name'].count().sort_values(ascending=False).head(10)
print(activity_by_time)

Series([], Name: Scientific_Name, dtype: int64)


# **Spatial Analysis**
***Location Type Hotspots***

In [ ]:
location_type_hotspots = df['Location_Type'].value_counts()
print(location_type_hotspots)

Location_Type
Forest       8546
Grassland    8531
Name: count, dtype: int64


***Plot-Level Analysis***

In [ ]:
plot_analysis = df['Plot_Name'].value_counts().head(10)
print(plot_analysis)

Plot_Name
MONO-0054    204
MONO-0094    118
MANA-0076    116
MONO-0102     88
MONO-0050     77
MONO-0045     73
ANTI-0188     72
ANTI-0124     71
MONO-0083     68
MANA-0120     63
Name: count, dtype: int64


# **Species analysis**
***Diversity Metrics***


In [ ]:
species_diversity = df.groupby('Location_Type')['Scientific_Name'].nunique()
print(species_diversity)

Location_Type
Forest       108
Grassland    107
Name: Scientific_Name, dtype: int64


***Activity Patterns***

In [ ]:
interval_counts = df['Interval_Length'].value_counts()
id_method_counts = df['ID_Method'].value_counts()
print("Interval Lengths:\n", interval_counts)
print("Identification Methods:\n", id_method_counts)

Interval Lengths:
 Interval_Length
0-2.5 min       8614
2.5 - 5 min     3410
5 - 7.5 min     2720
7.5 - 10 min    2333
Name: count, dtype: int64
Identification Methods:
 ID_Method
Singing          9865
Calling          4061
Visualization    3151
Name: count, dtype: int64


***Sex Ratio***

In [ ]:
sex_ratio = df['Sex'].value_counts()
print(sex_ratio)

Sex
Undetermined    13593
Male             3338
Female            146
Name: count, dtype: int64


#**Environmental Conditions**

***Temperature and Humidity Effect***

In [ ]:
temp_effect = df.groupby('Temperature')['Initial_Three_Min_Cnt'].mean().sort_values(ascending=False).head(10)
humidity_effect = df.groupby('Humidity')['Initial_Three_Min_Cnt'].mean().sort_values(ascending=False).head(10)
print("Temperature Impact on Bird Count:\n", temp_effect)
print("Humidity Impact on Bird Count:\n", humidity_effect)

Temperature Impact on Bird Count:
 Temperature
32.599998         1.0
35.000000    0.833333
13.200000    0.785714
33.900002    0.785714
26.100000    0.772093
11.600000        0.75
32.000000    0.714286
15.600000    0.711538
13.800000    0.703704
15.800000    0.692308
Name: Initial_Three_Min_Cnt, dtype: object
Humidity Impact on Bird Count:
 Humidity
48.000000         1.0
53.599998    0.909091
52.000000    0.884615
49.299999       0.875
69.500000    0.818182
75.300003    0.804124
89.199997         0.8
92.300003         0.8
38.500000    0.785714
41.200001        0.78
Name: Initial_Three_Min_Cnt, dtype: object


***Disturbance Effect***

In [ ]:
disturbance_effect = df.groupby('Disturbance')['Initial_Three_Min_Cnt'].mean()
print("Bird count by Disturbance level:\n", disturbance_effect)

Bird count by Disturbance level:
 Disturbance
Moderate effect on count    0.518417
No effect on count          0.532819
Serious effect on count     0.471816
Slight effect on count      0.547407
Name: Initial_Three_Min_Cnt, dtype: object


# **Distance and Behavior**

***Distance Analysis***

In [ ]:
distance_analysis = df.groupby('Distance')['Scientific_Name'].nunique().sort_values(ascending=False)
print(distance_analysis)

Distance
50 - 100 Meters    113
<= 50 Meters       110
Name: Scientific_Name, dtype: int64


***Flyover Frequency***

In [ ]:
flyover_counts = df['Flyover_Observed'].value_counts()
print("Flyover Behavior:\n", flyover_counts)

Flyover Behavior:
 Flyover_Observed
False    15591
True      1486
Name: count, dtype: int64


# **Observer Trends**

***Observer Bias***

In [ ]:
observer_bias = df['Observer'].value_counts().head(10)
print("Top 10 Observers by number of records:\n", observer_bias)

Top 10 Observers by number of records:
 Observer
Elizabeth Oswald    6391
Kimberly Serno      5902
Brian Swimelar      4784
Name: count, dtype: int64


***Visit Patterns***

In [ ]:
visit_patterns = df.groupby('Visit')['Scientific_Name'].nunique()
print("Species count by Visit number:\n", visit_patterns)

Species count by Visit number:
 Visit
1    119
2    100
3     74
Name: Scientific_Name, dtype: int64


# **Conservation Insights**

***Watchlist & Stewardship Status***

In [ ]:
watchlist_species = df[df['PIF_Watchlist_Status'] == True]['Scientific_Name'].value_counts().head(10)
stewardship_species = df[df['Regional_Stewardship_Status'] == True]['Scientific_Name'].value_counts().head(10)

print("Top 10 Watchlist Species:\n", watchlist_species)
print("Top 10 Stewardship Species:\n", stewardship_species)

Top 10 Watchlist Species:
 Scientific_Name
Hylocichla mustelina          309
Helmitheros vermivorus         31
Setophaga discolor             25
Setophaga cerulea               7
Oporornis formosus              2
Empidonax traillii              2
Vermivora cyanoptera            1
Melanerpes erythrocephalus      1
Name: count, dtype: int64
Top 10 Stewardship Species:
 Scientific_Name
Passerina cyanea           635
Contopus virens            574
Spizella pusilla           542
Empidonax virescens        462
Poecile carolinensis       376
Hylocichla mustelina       309
Pipilo erythrophthalmus    263
Picoides pubescens         263
Piranga olivacea           216
Setophaga citrina           64
Name: count, dtype: int64


***AOU Code Analysis***

In [ ]:
aou_counts = df['AOU_Code'].value_counts().head(10)
print("Top AOU Codes:\n", aou_counts)

Top AOU Codes:
 AOU_Code
NOCA    1160
CARW    1011
REVI     739
ETTI     728
INBU     635
EAWP     574
FISP     542
EUST     522
AMGO     509
AMRO     496
Name: count, dtype: int64


In [ ]:
# Save cleaned dataset
df.to_csv("cleaned_bird_data.csv", index=False)

# Download it to your computer
from google.colab import files
files.download("cleaned_bird_data.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>